In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import random

import utils

country_aggregated_df = pd.read_csv('./Data/Gas_Trade_Flows_IEA_202310 - Data.csv')

### Process data

In [ ]:
# Remove unnamed colum
country_aggregated_df = country_aggregated_df.drop(columns=['Unnamed: 2'])

# Remove the columns with flow data beteen Oct-08 and Aug-09
country_aggregated_df = country_aggregated_df.drop(columns=['Oct-08', 'Nov-08', 'Dec-08', 'Jan-09', 'Feb-09', 'Mar-09', 'Apr-09', 'May-09', 'Jun-09', 'Jul-09', 'Aug-09', 'Sep-09', 'Oct-09', 'Nov-09', 'Dec-09'])
country_aggregated_df.head()

In [ ]:
country_aggregated_df.shape

### Countries in the data

In [ ]:
unique_countries = set(country_aggregated_df['Exit'].unique()) | set(country_aggregated_df['Entry'].unique())

print("Data contains {} unique countries".format(len(unique_countries)))
print(unique_countries)


In [ ]:
print("Pure exporting countries: {}".format(set(country_aggregated_df['Exit'].unique()) - set(country_aggregated_df['Entry'].unique())))
print("Pure importing countries: {}".format(set(country_aggregated_df['Entry'].unique()) - set(country_aggregated_df['Exit'].unique())))

In [ ]:
# Retrieve columns with flow data 
mm_yyyy = country_aggregated_df.iloc[:,country_aggregated_df.columns.get_loc('Jan-10'):]

## Test configure grid for max-flow

In [ ]:
graphs = utils.create_graphs_from_dataset(country_aggregated_df)

In [ ]:
# Get most recent grid
oct_23 = graphs[-1]

In [ ]:
# Display edge data
oct_23_edges = utils.get_edge_data(oct_23)
oct_23_edges

In [ ]:
# Display node data
oct_23_nodes = utils.get_node_data(oct_23)
oct_23_nodes

## Convert multi-edge directed graph to directed graph
Required by max-flow algorithm

In [ ]:
def create_digraph_of(M):
    G = nx.DiGraph()
    for u,v,data in M.edges(data=True):
        w = data['flow'] if 'flow' in data else 0
        c = data['capacity'] if 'capacity' in data else 0
        if G.has_edge(u,v):
            G[u][v]['flow'] += w
            G[u][v]['capacity'] += c
        else:
            G.add_edge(u, v, flow=w, capacity=c)
    return G

oct_23_digraph = create_digraph_of(oct_23)

## Flow hierarchy
Network directionality, i.e., things move in one general direction

Source: https://web.mit.edu/~cmagee/www/documents/28-DetectingEvolvingPatterns_FlowHierarchy.pdf

Additional source: https://arxiv.org/pdf/1202.0191.pdf

"By definition, a pure random directed network embeds no hierarchy."

ISSUE: edges (pipelines) included in grid construction for a given year

In [ ]:
print("Flow degree, MultiDiGraph: ", nx.flow_hierarchy(oct_23, weight='flow'))
print("Flow degree, DiGraph: ", nx.flow_hierarchy(oct_23_digraph, weight='flow'))

In [ ]:
print("Flow degree, MultiDiGraph: ", nx.flow_hierarchy(utils.ER_benchmark(oct_23)))
print("Flow degree, DiGraph: ", nx.flow_hierarchy(utils.ER_benchmark(oct_23_digraph)))

In [ ]:
graph_names = [graphs[i].name for i in range(len(graphs))]
flow_hierarchy_multidigraph = [nx.flow_hierarchy(graph, weight="flow") for graph in graphs]
flow_hierarchy_digraph = [nx.flow_hierarchy(utils.create_digraph_of(graph), weight="flow") for graph in graphs]
flow_hierarchy_er_benchmark = [nx.flow_hierarchy(utils.ER_benchmark(graph)) for graph in graphs]
flow_hierarchy_er_benchmark_digraph = [nx.flow_hierarchy(utils.ER_benchmark(utils.create_digraph_of(graph))) for graph in graphs]

fig, axs = plt.subplots(1, 2, figsize=(20, 7))
axs[0].bar(graph_names, flow_hierarchy_multidigraph, label='Real', color='skyblue', alpha=0.8)
axs[0].bar(graph_names, flow_hierarchy_er_benchmark, label='ER', color='green', alpha=0.8)
axs[0].set_ylabel('Hierarchy degree')
axs[0].set_title('MultiDiGraph')
axs[0].set_ylim(0, 1)
axs[0].legend()
axs[0].set_xticks(range(0, len(graph_names), 4))
axs[0].set_xticklabels(graph_names[::4], rotation=45, fontsize=10)

axs[1].bar(graph_names, flow_hierarchy_digraph, label='Real', color='red', alpha=0.8)
axs[1].bar(graph_names, flow_hierarchy_er_benchmark_digraph, label='ER', color='orange', alpha=0.8)
axs[1].set_ylabel('Hierarchy degree')
axs[1].set_title('DiGraph')
axs[1].set_ylim(0, 1)
axs[1].legend()
axs[1].set_xticks(range(0, len(graph_names), 4))
axs[1].set_xticklabels(graph_names[::4], rotation=45, fontsize=10)


plt.tight_layout()

plt.show()


## Max flow

In [ ]:
# Define sources and sinks
sources = ['Algeria', 'Russia', 'Norway', 'Liquefied Natural Gas']
sinks = ['Germany', 'France', 'Italy']

In [ ]:
# Illustrative example
flow_value, flow_dict, flow_edges = utils.max_flow(oct_23_digraph, sources, sinks, capacity='capacity', show_plot=True)

### Explore load rate across MM-YYYY

In [ ]:
load_rates = []
for graph_data in graphs:
    graph = create_digraph_of(graph_data)
    
    # Calculate ideal and real flow values
    ideal_flow_value, _, _ = utils.max_flow(graph, sources, sinks, capacity='capacity', show_plot=False)
    real_flow_value, _, _ = utils.max_flow(graph, sources, sinks, capacity='flow', show_plot=False)

    # Calculate and append the load rate to the list
    load_rate = real_flow_value / ideal_flow_value
    load_rates.append(load_rate)

# Plotting
graph_names = [graphs[i].name for i in range(len(graphs))]
plt.figure(figsize=(30, 10))
plt.plot(load_rates, marker='o')
plt.ylabel('Load Rate', fontsize=15)
plt.title('Historic Load Rate of the European Natural Gas Grid', fontsize=20)
plt.xticks(range(0, len(graph_names), 4), [graph_names[i] for i in range(0, len(graph_names), 4)], rotation=45)
plt.show()


### Explore N-k analysis with max flow

In [82]:
sources = ['Algeria', 'Russia', 'Norway', 'Liquefied Natural Gas']
sinks = ['Germany', 'France', 'Italy']

def n_minus_k(G, sources, sinks, k_removals, capacity='capacity', heuristic='random', remove='edge'):

    results_df = pd.DataFrame(columns=['max_flow', 'max_flow_index', 'load_rate', 'removed_entity'])

    def add_super_source_sink(G, sources, sinks):
        super_source = "super_source"
        super_sink = "super_sink"

        G.add_node(super_source)
        for source in sources:
            G.add_edge(super_source, source, capacity=float('inf'))

        G.add_node(super_sink)
        for sink in sinks:
            G.add_edge(sink, super_sink, capacity=float('inf'))

        return super_source, super_sink
    
    source, sink = add_super_source_sink(G, sources, sinks)

    def get_initial_load_rate(G, sources, sinks):
        init_flow_value_real, _ = nx.maximum_flow(G, sources, sinks, capacity='flow', flow_func=nx.algorithms.flow.dinitz)
        init_flow_value_ideal, _ = nx.maximum_flow(G, sources, sinks, capacity='capacity', flow_func=nx.algorithms.flow.dinitz)
        return init_flow_value_real / init_flow_value_ideal
    
    initial_load_rate = get_initial_load_rate(G, source, sink)

    for k in range(k_removals):

        if heuristic == 'random':

            switch = True
            while switch:
                target = random.choice(list(G.nodes() if remove == 'node' else G.edges()))
                if target not in sources and target not in sinks and target != 'super_source' and target != 'super_sink':
                    switch = False

            G.remove_node(target) if remove == 'node' else G.remove_edge(*target)

        elif heuristic == 'greedy':
            return NotImplementedError
        
        elif heuristic == 'max_flow':
            # TODO: Implement max_flow heuristic
            flow_value, flow_dict = nx.maximum_flow(G, source, sink, capacity=capacity, flow_func=nx.algorithms.flow.dinitz)
            target = max(flow_dict.items(), key=lambda x: x[1])[0]
            
            G.remove_edge(*target)


        current_max_flow_value_ideal, flow_dict = nx.maximum_flow(G, source, sink, capacity='capacity', flow_func=nx.algorithms.flow.dinitz)
        current_max_flow_value_real, flow_dict = nx.maximum_flow(G, source, sink, capacity='flow', flow_func=nx.algorithms.flow.dinitz)

        current_load_rate = current_max_flow_value_real / current_max_flow_value_ideal

        results_df.loc[k] = [current_max_flow_value_ideal, current_load_rate / initial_load_rate, current_load_rate, target]

    return results_df

In [102]:
oct_23_digraph = create_digraph_of(oct_23)
results_df = n_minus_k(oct_23_digraph, sources, sinks, 10)
results_df

,max_flow,max_flow_index,load_rate,removed_entity
0,38277.44,1.000000,0.285913,"(Belgium, Luxembourg)"
1,38277.44,1.000000,0.285913,"(Slovak Republic, Ukraine)"
2,38277.44,1.000000,0.285913,"(Germany, Belgium)"
3,38277.44,1.000000,0.285913,"(Czech Republic, Austria)"
4,38277.44,1.000000,0.285913,"(Algeria, Spain)"
...,...,...,...,...
75,5187.00,1.516492,0.433584,"(Republic of Türkiye, Greece)"
76,5187.00,1.516492,0.433584,"(Spain, Morocco)"
77,4963.80,1.582568,0.452476,"(Liquefied Natural Gas, Croatia)"
78,4963.80,1.582568,0.452476,"(France, Spain)"
